#### NOTE: Maven dependencies in spark:
https://stackoverflow.com/questions/42978976/how-do-we-specify-maven-dependencies-in-pyspark



-  https://mvnrepository.com/
- https://spark.apache.org/docs/latest/configuration.html#available-properties


# NOTE: This code had errors.

In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName('jupyter-pyspark') \
    .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2")\
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2d22931b-a03e-4e97-bfd7-8449f94fc904;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 233ms :: artifacts dl 4ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central in [default]
	org.apache.kafka#kafka-clients;2.6.0 from central in [default]
	org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 from central in [default]
	org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 from central in [default]
	org.lz4#lz4-java;1.7.1 from central in [default]
	org.slf4j#s

In [3]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers","broker:29092") \
  .option("subscribe", "atm") \
  .option("includeHeaders", "true") \
  .load()
df2 = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "headers")

In [4]:
sq = df2.writeStream.format('console').queryName('this_query').start()
sq.awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+-------+
|key|value|headers|
+---+-----+-------+
+---+-----+-------+



21/11/14 18:17:18 ERROR Utils: Aborting task
java.lang.NoSuchMethodError: 'org.apache.kafka.clients.consumer.ConsumerRecords org.apache.kafka.clients.consumer.KafkaConsumer.poll(java.time.Duration)'
	at org.apache.spark.sql.kafka010.consumer.InternalKafkaConsumer.fetch(KafkaDataConsumer.scala:78)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.fetchData(KafkaDataConsumer.scala:539)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.fetchRecord(KafkaDataConsumer.scala:463)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.$anonfun$get$1(KafkaDataConsumer.scala:310)
	at org.apache.spark.util.UninterruptibleThread.runUninterruptibly(UninterruptibleThread.scala:77)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.runUninterruptiblyIfPossible(KafkaDataConsumer.scala:604)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.get(KafkaDataConsumer.scala:287)
	at org.apache.spark.sql.kafka010.KafkaBatchPartitionReader.next(KafkaBatchPartitio

StreamingQueryException: Writing job aborted.
=== Streaming Query ===
Identifier: this_query [id = 4a401d48-a5c5-4cfc-974f-45150dd43dcf, runId = 06a07043-4289-46d3-9596-33c89db9d50e]
Current Committed Offsets: {KafkaV2[Subscribe[atm]]: {"atm":{"0":683}}}
Current Available Offsets: {KafkaV2[Subscribe[atm]]: {"atm":{"0":684}}}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
WriteToMicroBatchDataSource ConsoleWriter[numRows=20, truncate=true]
+- Project [cast(key#8 as string) AS key#24, cast(value#9 as string) AS value#25, headers#15]
   +- StreamingDataSourceV2Relation [key#8, value#9, topic#10, partition#11, offset#12L, timestamp#13, timestampType#14, headers#15], org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaScan@1381cbb1, KafkaV2[Subscribe[atm]]


In [45]:
df = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers","broker:29092") \
  .option("subscribe", "atm") \
  .option("includeHeaders", "true") \
  .load()
df2 = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "headers")
df2.show()

21/11/14 18:14:22 ERROR Executor: Exception in task 0.0 in stage 2.0 (TID 2)
java.lang.NoSuchMethodError: 'org.apache.kafka.clients.consumer.ConsumerRecords org.apache.kafka.clients.consumer.KafkaConsumer.poll(java.time.Duration)'
	at org.apache.spark.sql.kafka010.consumer.InternalKafkaConsumer.fetch(KafkaDataConsumer.scala:78)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.fetchData(KafkaDataConsumer.scala:539)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.fetchRecord(KafkaDataConsumer.scala:463)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.$anonfun$get$1(KafkaDataConsumer.scala:310)
	at org.apache.spark.util.UninterruptibleThread.runUninterruptibly(UninterruptibleThread.scala:77)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.runUninterruptiblyIfPossible(KafkaDataConsumer.scala:604)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.get(KafkaDataConsumer.scala:287)
	at org.apache.spark.sql.kafka010.KafkaSourceRDD$$a

Py4JJavaError: An error occurred while calling o261.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (bf7c78b01d3d executor driver): java.lang.NoSuchMethodError: 'org.apache.kafka.clients.consumer.ConsumerRecords org.apache.kafka.clients.consumer.KafkaConsumer.poll(java.time.Duration)'
	at org.apache.spark.sql.kafka010.consumer.InternalKafkaConsumer.fetch(KafkaDataConsumer.scala:78)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.fetchData(KafkaDataConsumer.scala:539)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.fetchRecord(KafkaDataConsumer.scala:463)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.$anonfun$get$1(KafkaDataConsumer.scala:310)
	at org.apache.spark.util.UninterruptibleThread.runUninterruptibly(UninterruptibleThread.scala:77)
	at org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer.runUninterruptiblyIfPos

In [37]:
sq.isActive

True

In [38]:
dir(sq)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_jsq',
 'awaitTermination',
 'exception',
 'explain',
 'id',
 'isActive',
 'lastProgress',
 'name',
 'processAllAvailable',
 'recentProgress',
 'runId',
 'status',
 'stop']

21/11/14 18:08:34 ERROR MicroBatchExecution: Query this_query [id = f88319e1-a124-467c-b365-ca248af31c27, runId = 8f85733c-2bc0-4be4-a9d3-88cbeed583dc] terminated with error
org.apache.kafka.common.KafkaException: Failed to construct kafka consumer
	at org.apache.kafka.clients.consumer.KafkaConsumer.<init>(KafkaConsumer.java:718)
	at org.apache.kafka.clients.consumer.KafkaConsumer.<init>(KafkaConsumer.java:566)
	at org.apache.kafka.clients.consumer.KafkaConsumer.<init>(KafkaConsumer.java:549)
	at org.apache.spark.sql.kafka010.SubscribeStrategy.createConsumer(ConsumerStrategy.scala:107)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderConsumer.consumer(KafkaOffsetReaderConsumer.scala:82)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderConsumer.$anonfun$partitionsAssignedToConsumer$2(KafkaOffsetReaderConsumer.scala:533)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderConsumer.$anonfun$withRetriesWithoutInterrupt$1(KafkaOffsetReaderConsumer.scala:578)
	at scala.runtime.java8.JFunc